# Transfer Learning

Paper original: The influence of pattern similarity and transfer learning upon the training of a base perceptron B2 [Stevo Bozinovski y Ante Fulgosi]

Relacionado:
https://pdfs.semanticscholar.org/d6b9/d3de85a43f719be3973ce9ca289d89bc5224.pdf?_ga=2.144951024.554554672.1635970300-1836405539.1635970300

### Ideas principales - Cambios x 2

* Las capas de extracción de características nos puede servir para otras tareas.
* Usar un modelo ya entrenado.
* Congelar los pesos sinapticos: entrenar la(s) última(s) capa(s).

**¿Cómo se cuántas capas *freezar*?**

Hay que hacer la tarea...

https://cs.stanford.edu/people/karpathy/deepvideo/deepvideo_cvpr2014.pdf

### Recapitulando

El aprendizaje por transferencia es una técnica de aprendizaje automático en la que un modelo entrenado en una tarea se reorienta en una segunda tarea relacionada.

Utilizar esta técnica puede acelerar el entrenamiento y precisión de nuestro modelo, y con menos datos de entrenamiento.

**Dato de color:**
- En 1976 Stevo Bozinovski y Ante Fulgosi publican paper que estudiaba el aprendizaje por transferencia en el entrenamiento de redes neuronales. 
- En 1981 se aplicó y demostró el transfer learning en el entrenamiento de una red neuronal en un dataset de imágenes que eran letras de terminales de PC.



**A continuación se proveen algunas situaciones y recomendaciones (de lo que funciona en la práctica):**

* **Cuando nuestro dataset es pequeño y similar al original (con el que fue entrenada la red A):** entrenar solo la última capa completamente conectada.

* **Cuando nuestro dataset es pequeño y diferente al original:** entrenar solo las capas completamente conectadas.

* **Si nuestro conjunto de datos es *big* y similar al original:** congelar las primeras capas (características simples) y entrenar el resto de las capas.

* **Si nuestro conjunto de datos es *big* y diferente al original:** entrenar el modelo desde cero y reutilizar la arquitectura de red (usando los pesos entrenados como punto inicial).

<img src="images/transfer_learning.png" alt="Drawing" style="width: 600px;"/>

### ¿Qué podemos plantear de las situaciones planteadas previamente?

Que el **tamaño de nuestro conjunto de datos** y la **similitud** que tenga **con el dataset original** son las dos claves a considerar antes de aplicar el aprendizaje por transferencia.

# Veamos un ejemplo

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [ ]:
# transformacions para usar con el modelo resnet18 entrenado con image net
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'my_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x]) for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, 
                                              shuffle=True, num_workers=4) for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

# por si se quiere usar en colab con GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class_names

In [ ]:
def imshow(inp, title=None):

    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    
    # llevado al intervalo [0, 1]
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) 

inputs, classes = next(iter(dataloaders['train']))

# grid
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoca {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # solo se hace update de gradiente si estamos en fase de training
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # nos vamos quedando con el mejor modelo en caso de que empeore en epocas futuras
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    print('Mejor accuracy: {:4f}'.format(best_acc))

    # retornar mejor modelos
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('Prediccion: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

## En este caso solo ajustamos la cantidad de clases y entrenamos partiendo de los pesos ajustados previamente

In [ ]:
model_ft = models.resnet18(pretrained=True)

In [ ]:
#model_ft.conv1.weight

In [ ]:
num_ftrs = model_ft.fc.in_features

In [ ]:
num_ftrs

In [ ]:
# model_ft.fc

In [ ]:
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# reducir en un factor de 0.1 cada 7 epocas
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
visualize_model(model_ft)


In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=2)

In [ ]:
visualize_model(model_ft)


## En este caso solo vamos a entrenar la capa final

Esto llevará mucho menos tiempo que el caso anterior, ya que no es necesario calcular los gradientes para la mayor parte de la red. Sin embargo, es necesario calcular la propagación hacia adelante.

In [ ]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# reducir en un factor de 0.1 cada 7 epocas
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [ ]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=2)

In [ ]:
visualize_model(model_conv)
